In [1]:
import os
import pyupbit
import datetime
import pandas as pd
import sqlite3
import time

def create_table(con):
    cursor = con.cursor()
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS ticker_data (
        type TEXT,
        code TEXT,
        opening_price REAL,
        high_price REAL,
        low_price REAL,
        trade_price REAL,
        trade_volume REAL,
        trade_date TEXT,
        trade_time TEXT,
        trade_timestamp INTEGER
    )
    ''')
    con.commit()

def save_data(con, df):
    try:
        df.to_sql(name='ticker_data', con=con, if_exists='append', index=False)
        print("Data saved successfully.")
    except Exception as e:
        print(f"Error saving data: {e}")

def fetch_data(ticker, start, end):
    all_data = []
    current = start
    while current < end:
        next_current = min(current + datetime.timedelta(days=200), end)
        to_time = next_current - datetime.timedelta(hours=9)  # UTC로 변환
        df = pyupbit.get_ohlcv(ticker, interval='day', to=to_time.strftime('%Y-%m-%d %H:%M:%S'), count=200)
        if df is not None and not df.empty:
            all_data.append(df)
        current = next_current
        time.sleep(0.1)  # API 호출 제한 방지
    
    if all_data:
        combined_df = pd.concat(all_data)
        return combined_df[(combined_df.index >= start) & (combined_df.index < end)]
    return pd.DataFrame()

if __name__ == "__main__":
    ticker = "KRW-BTC"  # 솔라나 티커
    start_date = datetime.datetime(2024, 12, 1, 0, 0)
    end_date = datetime.datetime(2025, 1, 9, 0, 0)  # 12월 1일까지 포함하기 위해

    db_file = "btc_day_20241201_20250108.db"

    # 기존 데이터베이스 파일 삭제 (새로 생성하기 위해)
    if os.path.exists(db_file):
        os.remove(db_file)

    con = sqlite3.connect(db_file)
    create_table(con)

    current_date = start_date
    while current_date < end_date:
        next_date = min(current_date + datetime.timedelta(days=1), end_date)

        # 데이터 fetch
        df = fetch_data(ticker, current_date, next_date)
        if df is not None and not df.empty:
            df['type'] = 'ticker'  # 타입 추가
            df['code'] = ticker  # 코드 추가
            df['trade_timestamp'] = df.index.astype('int64') // 10**9  # 타임스탬프 추가 (초 단위)
            # 날짜와 시간 컬럼 추가
            df['trade_date'] = df.index.date.astype(str)  # YYYY-MM-DD 형식
            df['trade_time'] = df.index.strftime('%H:%M')  # HH:MM 형식

            # 데이터베이스에 저장
            save_data(con, df[['type', 'code', 'open', 'high', 'low', 'close', 'volume', 
                                'trade_date', 'trade_time', 'trade_timestamp']].rename(columns={
                'open': 'opening_price',
                'high': 'high_price',
                'low': 'low_price',
                'close': 'trade_price',
                'volume': 'trade_volume'
            }))
        else:
            print(f"No data found for {current_date.date()}")

        print(f"Completed processing for {current_date.date()}")

        current_date = next_date
        time.sleep(1)  # API 요청 제한을 피하기 위한 대기 시간

    con.close()
    print("Data collection completed for all dates.")


Data saved successfully.
Completed processing for 2024-12-01
Data saved successfully.
Completed processing for 2024-12-02
Data saved successfully.
Completed processing for 2024-12-03
Data saved successfully.
Completed processing for 2024-12-04
Data saved successfully.
Completed processing for 2024-12-05
Data saved successfully.
Completed processing for 2024-12-06
Data saved successfully.
Completed processing for 2024-12-07
Data saved successfully.
Completed processing for 2024-12-08
Data saved successfully.
Completed processing for 2024-12-09
Data saved successfully.
Completed processing for 2024-12-10
Data saved successfully.
Completed processing for 2024-12-11
Data saved successfully.
Completed processing for 2024-12-12
Data saved successfully.
Completed processing for 2024-12-13
Data saved successfully.
Completed processing for 2024-12-14
Data saved successfully.
Completed processing for 2024-12-15
Data saved successfully.
Completed processing for 2024-12-16
Data saved successfully.

: 